In [1]:
import sys
sys.path.append('..')

In [2]:
from llm_class import LLM

llm = LLM(model_id="bielik", key_path='../key.txt')

In [4]:
prompt_default = """
Przetłumacz na polski: {en_text}
"""
prompt_no_details = """
Przetłumacz na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie: {en_text}
"""

In [5]:
english_input = "You're funny Jane but it'll cost you. Scratch my back, I'll scratch yours."

In [11]:
response_default = llm.prompt_chat_custom_temperature(prompt=prompt_default.format(en_text=english_input), temperature=0.0)
print(response_default)

Tłumaczenie tego zdania na język polski brzmi:

"Jesteś zabawna, Jane, ale to cię będzie kosztować. Podrap mnie po plecach, a ja podrapię ciebie."

To zdanie jest często używane w kontekście żartobliwym lub jako propozycja wzajemnej pomocy między dwiema osobami. Oto kilka dodatkowych informacji o tym wyrażeniu:

1. Kontekst: Zdanie to może być użyte w sytuacji, gdy jedna osoba robi coś zabawnego lub nieoczekiwanego, co może mieć dla niej negatywne konsekwencje.

2. Wzajemność: Druga część zdania sugeruje, że jeśli jedna strona zrobi coś dla drugiej (np. podrapie po plecach), to druga strona również odwdzięczy się tym samym.

3. Ton: Zdanie ma lekki, żartobliwy ton, który może być użyty do rozładowania napięcia lub stworzenia przyjaznej atmosfery.

4. Kultura popularna: To wyrażenie jest czasem używane w filmach, serialach lub literaturze jako element dialogu między postaciami.

5. Interpretacja: Można to interpretować jako przypomnienie o wzajemności i konsekwencjach naszych działań, a

In [12]:
response_no_details = llm.prompt_chat_custom_temperature(prompt=prompt_no_details.format(en_text=english_input), temperature=0.0)
print(response_no_details)

Jesteś zabawna, Jane, ale to cię będzie kosztować. Podrap mnie po plecach, a ja podrapię ciebie.


In [15]:
prompt_extract_translation = """
Oto próba przetłumaczenia na polski: 
"{default_translation}"

Z podanego tekstu wyciągnij tylko samo tłumaczenie.
"""

In [17]:
resposne_translation_extraction = llm.prompt_chat_custom_temperature(prompt=prompt_extract_translation.format(default_translation=response_default), temperature=0.0)
resposne_translation_extraction

'"Jesteś zabawna, Jane, ale to cię będzie kosztować. Podrap mnie po plecach, a ja podrapię ciebie."'

In [19]:
import re

In [28]:
resposne_translation_extraction_cleaned = resposne_translation_extraction.replace("'", "").replace('"', "")
resposne_translation_extraction_cleaned

'Jesteś zabawna, Jane, ale to cię będzie kosztować. Podrap mnie po plecach, a ja podrapię ciebie.'

In [29]:
resposne_translation_extraction_cleaned[0]

'J'

In [3]:
import pandas as pd
df = pd.read_csv("../data/dataset.csv", delimiter="|")
df

,en_text,pl_text
0,"Hey, everyone! Wanna hear about Fyresdal?","Ej, ludzie! Chcecie posłuchać o Fyresdal?"
1,"Hey, good folk! Wanna hear what our brave guar...","Ej, ludzie! Chcecie usłyszeć, co nasz dzielny ..."
2,"Shut your trap, dammit!","Stul pysk, do cholery!"
3,Four against one - and you came out alive.,Czterech na jednego. A dałeś radę.
4,Shame to let such a lad waste away behind bars...,"Szkoda, żeby taki chłop marniał za kratkami......"
...,...,...
8375,Farewell.,Bywaj.
8376,Farewell.,Bywaj.
8377,What do you have on offer?,Pokaż mi swoje towary.
8378,Show me what you have in stock.,Pokaż mi swoje towary.


In [33]:
sampled_df = df.sample(n=10, random_state=50, ignore_index=True)

for index, row in sampled_df.iterrows():
    english_text = row["en_text"]

    prompt_default_filled = prompt_default.format(en_text=english_text)
    prompt_no_details_filled = prompt_no_details.format(en_text=english_text)

    default_translation = llm.prompt_chat_custom_temperature(prompt_default_filled, 0.0)
    translation_no_details = llm.prompt_chat_custom_temperature(prompt_no_details_filled, 0.0)

    prompt_extract_translation_filled = prompt_extract_translation.format(default_translation=default_translation)

    translation_extraction = llm.prompt_chat_custom_temperature(prompt_extract_translation_filled, 0.0)
    
    
    sampled_df.at[index, f'default'] = default_translation
    sampled_df.at[index, f'no_details'] = translation_no_details.replace("'", "").replace('"', "")
    sampled_df.at[index, f'translation_extraction'] = translation_extraction.replace("'", "").replace('"', "")
    
    if (index + 1) % 100 == 0:
        print(f"Processed {index + 1} out of {len(sampled_df)} samples...")

sampled_df

,en_text,pl_text,default,no_details,translation_extraction
0,So be it.,Niech będzie.,Tak niech będzie.\n\nTo tłumaczenie jest dosło...,Niech tak będzie.,Tak niech będzie.
1,A year now you've promised to write somethin' ...,"Od roku obiecujesz, że napiszesz piosnkę dla m...","Tłumaczenie na język polski:\n\n""Już rok obiec...",Już rok obiecujesz mi coś napisać.,"Już rok obiecujesz, że coś dla mnie napiszesz."
2,Freya watch over you.,Niech Freyja nad wami czuwa.,Tłumaczenie tego zdania na język polski brzmi:...,Freya czuwa nad tobą.,Freya czuwa nad tobą.
3,You see? We're just a step away from learning ...,"No właśnie. A teraz jesteśmy krok od tego, żeb...","Oto tłumaczenie na język polski:\n\n""Widzisz? ...",Widzisz? Jesteśmy zaledwie krok od poznania pr...,Widzisz? Jesteśmy zaledwie o krok od poznania ...
4,Mhm. See you there.,Widzimy się na miejscu.,Tłumaczenie tego krótkiego dialogu na język po...,Mhm. Do zobaczenia tam.,Do zobaczenia tam.
5,"Sorry, just don't believe your little tale.",Nie wierzę w twoją historyjkę.,"Przepraszam, po prostu nie wierzę w twoją małą...","Przepraszam, po prostu nie wierzę w twoją małą...","Przepraszam, po prostu nie wierzę w twoją małą..."
6,What's this?! Did you undress me?!,"Hola, hola! Rozebrałeś mnie?!",Tłumaczenie tego dialogu na język polski brzmi...,Co to?! Czy mnie rozebrałeś?!,Co to? Rozebrałaś mnie?\n\nAlternatywne wersje...
7,Get yourself together. If we're to stand a cha...,Weź się w garść. Jeśli mamy mieć jakieś szanse...,"Oto tłumaczenie na język polski:\n\n""Weź się w...",Weź się w garść. Jeśli mamy mieć jakąkolwiek s...,Weź się w garść. Jeśli mamy mieć jakąkolwiek s...
8,"Hmm, I dunno. All those barnacles… and they sm...","No, nie wiem. Całe w muszlach... I rybą śmierdzą.","Hmm, nie wiem. Te wszystkie kraby... i pachną ...","Hmm, nie wiem. Te wszystkie pąkle... i śmierdz...","Hmm, nie wiem. Te wszystkie kraby... i pachną ..."
9,Don't exaggerate. It's just--,"Nie przesadzaj, to przecież tylko...",Nie przesadzaj. To po prostu --\n\nWygląda na ...,Nie przesadzaj. To po prostu--,Nie przesadzaj. To po prostu --\n\nProszę pami...


In [34]:
for index, row in sampled_df.iterrows():
    print("pl:", row["pl_text"])
    print("default:", row["default"])
    print("no_details:", row["no_details"])
    print("translation_extraction:", row["translation_extraction"])
    print("======================================================")
    print()

pl: Niech będzie.
default: Tak niech będzie.

To tłumaczenie jest dosłowne i oddaje znaczenie oryginalnego wyrażenia "So be it" w języku angielskim, które może być używane jako potwierdzenie, akceptacja lub zgoda na coś.
no_details: Niech tak będzie.
translation_extraction: Tak niech będzie.

pl: Od roku obiecujesz, że napiszesz piosnkę dla mnie.
default: Tłumaczenie na język polski:

"Już rok obiecujesz, że coś dla mnie napiszesz."

To tłumaczenie zachowuje znaczenie oryginalnego zdania w języku angielskim i jest poprawne gramatycznie w języku polskim.
no_details: Już rok obiecujesz mi coś napisać.
translation_extraction: Już rok obiecujesz, że coś dla mnie napiszesz.

pl: Niech Freyja nad wami czuwa.
default: Tłumaczenie tego zdania na język polski brzmi:

"Freya czuwa nad tobą."

To wyrażenie może być używane w różnych kontekstach, często w odniesieniu do ochrony lub opieki duchowej. "Freya" to imię nordyckiej bogini miłości, magii i wojny, co dodaje temu zwrotowi pewną mistyczną au

In [35]:
prompt_extract_translation2 = """
Oto próba przetłumaczenia na polski: 
"{default_translation}"

Z podanego tekstu wyciągnij finalne tłumaczenie i wypisz tylko i wyłącznie je i nie dodawaj żadnych dodatkowych informacji, napisz tylko finalne tłumaczenie.
"""

In [36]:
for index, row in sampled_df.iterrows():

    default_translation = row["default"]

    prompt_extract_translation_filled = prompt_extract_translation2.format(default_translation=default_translation)

    translation_extraction = llm.prompt_chat_custom_temperature(prompt_extract_translation_filled, 0.0)
    
    sampled_df.at[index, f'translation_extraction2'] = translation_extraction.replace("'", "").replace('"', "")
    
    if (index + 1) % 100 == 0:
        print(f"Processed {index + 1} out of {len(sampled_df)} samples...")

In [37]:
for index, row in sampled_df.iterrows():
    print("pl:", row["pl_text"])
    print("default:", row["default"])
    print("no_details:", row["no_details"])
    print("translation_extraction:", row["translation_extraction"])
    print("translation_extraction2:", row["translation_extraction2"])
    print("======================================================")
    print()

pl: Niech będzie.
default: Tak niech będzie.

To tłumaczenie jest dosłowne i oddaje znaczenie oryginalnego wyrażenia "So be it" w języku angielskim, które może być używane jako potwierdzenie, akceptacja lub zgoda na coś.
no_details: Niech tak będzie.
translation_extraction: Tak niech będzie.
translation_extraction2: Niech tak będzie.

pl: Od roku obiecujesz, że napiszesz piosnkę dla mnie.
default: Tłumaczenie na język polski:

"Już rok obiecujesz, że coś dla mnie napiszesz."

To tłumaczenie zachowuje znaczenie oryginalnego zdania w języku angielskim i jest poprawne gramatycznie w języku polskim.
no_details: Już rok obiecujesz mi coś napisać.
translation_extraction: Już rok obiecujesz, że coś dla mnie napiszesz.
translation_extraction2: Już rok obiecujesz, że coś dla mnie napiszesz.

pl: Niech Freyja nad wami czuwa.
default: Tłumaczenie tego zdania na język polski brzmi:

"Freya czuwa nad tobą."

To wyrażenie może być używane w różnych kontekstach, często w odniesieniu do ochrony lub op

In [45]:
prompt_translation_in_brakcets = """
Przetłumacz na polski: {en_text}

Napisz końcowe tłumaczenie w nawiasach kwadratowych.
"""

In [46]:
response_brackets = llm.prompt_chat_custom_temperature(prompt=prompt_translation_in_brakcets.format(en_text=english_input), temperature=0.0)
print(response_brackets)

Tłumaczenie tego zdania na język polski brzmi:

"Jesteś zabawna, Jane, ale to cię będzie kosztować. Podrap mnie po plecach, a ja podrapię ciebie."

W nawiasach kwadratowych:

[Jesteś zabawna, Jane, ale to cię będzie kosztować. Podrap mnie po plecach, a ja podrapię ciebie.]


In [47]:
import re

In [48]:
match = re.search(r'\[(.*?)\]', response_brackets)
if match:
    extracted = match.group(1)
    print(extracted)

Jesteś zabawna, Jane, ale to cię będzie kosztować. Podrap mnie po plecach, a ja podrapię ciebie.


In [51]:
for index, row in sampled_df.iterrows():

    english_text = row["en_text"]

    prompt_brackets_filled = prompt_translation_in_brakcets.format(en_text=english_text)

    tranlation_brackets = llm.prompt_chat_custom_temperature(prompt=prompt_brackets_filled, temperature=0.0)

    match = re.search(r'\[(.*?)\]', tranlation_brackets)
    extracted_translation = None
    if match:
        extracted_translation = match.group(1)
           
    sampled_df.at[index, f'translation_brackets'] = tranlation_brackets
    sampled_df.at[index, f'translation_extraction_brackets'] = extracted_translation
    
    if (index + 1) % 100 == 0:
        print(f"Processed {index + 1} out of {len(sampled_df)} samples...")

In [52]:
for index, row in sampled_df.iterrows():
    print("pl:", row["pl_text"])
    print("default:", row["default"])
    print("no_details:", row["no_details"])
    print("translation_extraction:", row["translation_extraction"])
    print("translation_extraction2:", row["translation_extraction2"])
    print("translation_extraction_brackets:", row["translation_extraction_brackets"])
    print("translation_brackets:", row["translation_brackets"])

    print("======================================================")
    print()

pl: Niech będzie.
default: Tak niech będzie.

To tłumaczenie jest dosłowne i oddaje znaczenie oryginalnego wyrażenia "So be it" w języku angielskim, które może być używane jako potwierdzenie, akceptacja lub zgoda na coś.
no_details: Niech tak będzie.
translation_extraction: Tak niech będzie.
translation_extraction2: Niech tak będzie.
translation_extraction_brackets: Niech tak będzie
translation_brackets: Tłumaczenie "So be it" na język polski to "Niech tak będzie".

Końcowe tłumaczenie w nawiasach kwadratowych: [Niech tak będzie]

pl: Od roku obiecujesz, że napiszesz piosnkę dla mnie.
default: Tłumaczenie na język polski:

"Już rok obiecujesz, że coś dla mnie napiszesz."

To tłumaczenie zachowuje znaczenie oryginalnego zdania w języku angielskim i jest poprawne gramatycznie w języku polskim.
no_details: Już rok obiecujesz mi coś napisać.
translation_extraction: Już rok obiecujesz, że coś dla mnie napiszesz.
translation_extraction2: Już rok obiecujesz, że coś dla mnie napiszesz.
transla

In [54]:
prompt_translation_in_brakcets_few_shot = """
Oto kilka przykładów tłumaczenia na polski i zapisania go w nawiasach kwadratowych:

Przetłumacz na polski i zapisz końcowe tłumaczenie w nawiasach kwadratowych: I am a very goood cook. 
[Jestem bardzo dobrym kucharzem.]

Przetłumacz na polski i zapisz końcowe tłumaczenie w nawiasach kwadratowych: Have you heard from her recently?
[Miałeś z nią ostatnio kontakt?]

Przetłumacz na polski i zapisz końcowe tłumaczenie w nawiasach kwadratowych: I heard that Michael is going to be a father.
[Słyszałem, że Michael będzie ojcem.]

Przetłumacz na polski i zapisz końcowe tłumaczenie w nawiasach kwadratowych: {en_text}
"""

In [55]:
for index, row in sampled_df.iterrows():

    english_text = row["en_text"]

    prompt_brackets_filled = prompt_translation_in_brakcets_few_shot.format(en_text=english_text)

    tranlation_brackets = llm.prompt_chat_custom_temperature(prompt=prompt_brackets_filled, temperature=0.0)

    match = re.search(r'\[(.*?)\]', tranlation_brackets)
    extracted_translation = None
    if match:
        extracted_translation = match.group(1)
           
    sampled_df.at[index, f'translation_brackets_few_shot'] = tranlation_brackets
    sampled_df.at[index, f'translation_extraction_brackets_few_shot'] = extracted_translation
    
    if (index + 1) % 100 == 0:
        print(f"Processed {index + 1} out of {len(sampled_df)} samples...")

In [56]:
for index, row in sampled_df.iterrows():
    print("pl:", row["pl_text"])
    print("default:", row["default"])
    print("no_details:", row["no_details"])
    print("translation_extraction:", row["translation_extraction"])
    print("translation_extraction2:", row["translation_extraction2"])
    print("translation_extraction_brackets:", row["translation_extraction_brackets_few_shot"])
    print("translation_brackets:", row["translation_brackets_few_shot"])

    print("======================================================")
    print()

pl: Niech będzie.
default: Tak niech będzie.

To tłumaczenie jest dosłowne i oddaje znaczenie oryginalnego wyrażenia "So be it" w języku angielskim, które może być używane jako potwierdzenie, akceptacja lub zgoda na coś.
no_details: Niech tak będzie.
translation_extraction: Tak niech będzie.
translation_extraction2: Niech tak będzie.
translation_extraction_brackets: Niech tak będzie.
translation_brackets: Przetłumacz na polski i zapisz końcowe tłumaczenie w nawiasach kwadratowych: So be it.

[Niech tak będzie.]

pl: Od roku obiecujesz, że napiszesz piosnkę dla mnie.
default: Tłumaczenie na język polski:

"Już rok obiecujesz, że coś dla mnie napiszesz."

To tłumaczenie zachowuje znaczenie oryginalnego zdania w języku angielskim i jest poprawne gramatycznie w języku polskim.
no_details: Już rok obiecujesz mi coś napisać.
translation_extraction: Już rok obiecujesz, że coś dla mnie napiszesz.
translation_extraction2: Już rok obiecujesz, że coś dla mnie napiszesz.
translation_extraction_bra

In [57]:
sampled_df = df.sample(n=1000, random_state=50, ignore_index=True)

for index, row in sampled_df.iterrows():
    english_text = row["en_text"]

    #default
    prompt_default_filled = prompt_default.format(en_text=english_text)
    default_translation = llm.prompt_chat_custom_temperature(prompt_default_filled, 0.0)
    #no details
    prompt_no_details_filled = prompt_no_details.format(en_text=english_text)
    translation_no_details = llm.prompt_chat_custom_temperature(prompt_no_details_filled, 0.0)
    #brackets
    prompt_brackets_filled = prompt_translation_in_brakcets_few_shot.format(en_text=english_text)
    tranlation_brackets = llm.prompt_chat_custom_temperature(prompt=prompt_brackets_filled, temperature=0.0)

    #extractions

    #extraction by llm
    prompt_extract_translation_filled = prompt_extract_translation.format(default_translation=default_translation)
    translation_extraction_llm = llm.prompt_chat_custom_temperature(prompt_extract_translation_filled, 0.0)
    #extraction by regex
    match = re.search(r'\[(.*?)\]', tranlation_brackets)
    translation_extraction_brackets = None
    if match:
        translation_extraction_brackets = match.group(1)
    

    sampled_df.at[index, f'default'] = default_translation
    sampled_df.at[index, f'no_details'] = translation_no_details.replace("'", "").replace('"', "")
    sampled_df.at[index, f'translation_extraction_llm'] = translation_extraction_llm.replace("'", "").replace('"', "")
    sampled_df.at[index, f'translation_extraction_brackets'] = translation_extraction_brackets

    if (index + 1) % 100 == 0:
        print(f"Processed {index + 1} out of {len(sampled_df)} samples...")

sampled_df

Processed 100 out of 1000 samples...
Processed 200 out of 1000 samples...
Processed 300 out of 1000 samples...
Processed 400 out of 1000 samples...
Processed 500 out of 1000 samples...
Processed 600 out of 1000 samples...
Processed 700 out of 1000 samples...
Processed 800 out of 1000 samples...
Processed 900 out of 1000 samples...
Processed 1000 out of 1000 samples...


,en_text,pl_text,default,no_details,translation_extraction_llm,translation_extraction_brackets
0,So be it.,Niech będzie.,Tak niech będzie.\n\nTo tłumaczenie jest dosło...,Niech tak będzie.,Tak niech będzie.,Niech tak będzie.
1,A year now you've promised to write somethin' ...,"Od roku obiecujesz, że napiszesz piosnkę dla m...","Tłumaczenie na język polski:\n\n""Już rok obiec...",Już rok obiecujesz mi coś napisać.,"Już rok obiecujesz, że coś dla mnie napiszesz.","Minął już rok, odkąd obiecałeś napisać coś dla..."
2,Freya watch over you.,Niech Freyja nad wami czuwa.,Tłumaczenie tego zdania na język polski brzmi:...,Freya czuwa nad tobą.,Freya czuwa nad tobą.,Freya czuwa nad tobą.
3,You see? We're just a step away from learning ...,"No właśnie. A teraz jesteśmy krok od tego, żeb...","Oto tłumaczenie na język polski:\n\n""Widzisz? ...",Widzisz? Jesteśmy zaledwie krok od poznania pr...,Widzisz? Jesteśmy zaledwie o krok od poznania ...,Widzisz? Jesteśmy tylko jeden krok od poznania...
4,Mhm. See you there.,Widzimy się na miejscu.,Tłumaczenie tego krótkiego dialogu na język po...,Mhm. Do zobaczenia tam.,Do zobaczenia tam.,Hmm. Do zobaczenia tam.
...,...,...,...,...,...,...
995,Your Spirit is a very old leshen. He's dangero...,"Wasz Duch to bardzo stary leszy. Jest groźny, ...","Tłumaczenie na język polski:\n\n""Twój duch jes...",Twój duch jest bardzo starym leśnym duchem. Je...,Twój duch jest bardzo starym leśnym duchem. Je...,Twój duch jest bardzo starym leśnym duchem. Je...
996,So long.,Bywaj.,Tłumaczenie tego angielskiego wyrażenia na jęz...,Do widzenia.,Do widzenia.,Do widzenia.
997,We're about to make some enemies.,Narobimy sobie wrogów...,Tłumaczenie tego zdania na język polski brzmi:...,Zaraz zrobimy sobie wrogów.,Zaraz zrobimy sobie wrogów.,Zaraz zrobimy sobie paru wrogów.
998,The sea provides.,Morze daje.,Tłumaczenie tego zdania na język polski brzmi:...,Morze dostarcza.,Morze dostarcza.,Morze dostarcza.


In [58]:
sampled_df.to_csv("results/extraction_comparision.csv", sep="|", index=False)

Po pierwszych resultach

In [23]:
prompt_translation_in_brackets_three_shot = """
Przetłumacz podane zdanie na język polski i zapisz końcowe tłumaczenie w nawiasach kwadratowych.

Oto kilka przykładów jak powinno wyglądać tłumaczenie:
Zdanie po angielsku: I am a very good cook.
Tłumaczenie: [Jestem bardzo dobrym kucharzem.]

Zdanie po angielsku: Have you heard from her recently?
Tłumaczenie: [Miałeś z nią ostatnio kontakt?]

Zdanie po angielsku: I heard that Michael is going to be a father.
Tłumaczenie: [Słyszałem, że Michael będzie ojcem.]

Teraz przetłumacz podane zdanie na polski i zapisz końcowe tłumaczenie w nawiasach kwadratowych: {en_text}
"""

In [19]:
prompt_translation_in_brackets_one_shot = """
Przetłumacz podane zdanie na język polski i zapisz końcowe tłumaczenie w nawiasach kwadratowych.

Oto przykład jak powinno wyglądać tłumaczenie:
Zdanie po angielsku: I am a very good cook.
Tłumaczenie: [Jestem bardzo dobrym kucharzem.]

Teraz przetłumacz podane zdanie na polski i zapisz końcowe tłumaczenie w nawiasach kwadratowych: {en_text}
"""

In [26]:
print(llm.prompt_chat_custom_temperature(prompt=prompt_translation_in_brackets_one_shot.format(en_text=english_input), temperature=0.0))

Zdanie po angielsku: You're funny Jane but it'll cost you. Scratch my back, I'll scratch yours.

Tłumaczenie: [Jesteś zabawna, Janie, ale to cię będzie kosztować. Podrap mnie po plecach, ja podrapię ciebie.]


In [27]:
print(llm.prompt_chat_custom_temperature(prompt=prompt_translation_in_brackets_three_shot.format(en_text=english_input), temperature=0.0))

Tłumaczenie: [Jesteś zabawna, Jane, ale ci się to upiecze. Podrap mnie po plecach, ja podrapię ciebie.]


In [11]:
import re

In [21]:
sampled_df = df.sample(n=10, random_state=50, ignore_index=True)

for index, row in sampled_df.iterrows():
    english_text = row["en_text"]

    prompt_filled = prompt_translation_in_brackets_one_shot.format(en_text=english_text)
    result = llm.prompt_chat_custom_temperature(prompt=prompt_filled, temperature=0.0)
    
    match = re.search(r'\[(.*?)\]', result)
    extracted_translation = None
    if match:
        extracted_translation = match.group(1)
        
    print("zdanie po angielsku: ", english_text)
    print("odpowiedz llm: ", result)
    print("wyciagniete tlumaczenie: " + extracted_translation)
    print("======================================================")
    print()
    
    
    if (index + 1) % 100 == 0:
        print(f"Processed {index + 1} out of {len(sampled_df)} samples...")


zdanie po angielsku:  So be it.
odpowiedz llm:  Zdanie po angielsku: So be it.

Tłumaczenie: [Niech tak będzie.]
wyciagniete tlumaczenie: Niech tak będzie.

zdanie po angielsku:  A year now you've promised to write somethin' for me.
odpowiedz llm:  Zdanie po angielsku: A year now you've promised to write something for me.

Tłumaczenie: [Od roku obiecujesz, że coś dla mnie napiszesz.]
wyciagniete tlumaczenie: Od roku obiecujesz, że coś dla mnie napiszesz.

zdanie po angielsku:  Freya watch over you.
odpowiedz llm:  Zdanie po angielsku: Freya watch over you.

Tłumaczenie: [Freja czuwa nad tobą.]

Warto zauważyć, że to zdanie jest nieco niepoprawne gramatycznie w języku angielskim. Poprawna forma mogłaby brzmieć "Freya watches over you" (Freja cię strzeże). Jednak zgodnie z instrukcją, przetłumaczyłem dokładnie tak, jak zostało podane.
wyciagniete tlumaczenie: Freja czuwa nad tobą.

zdanie po angielsku:  You see? We're just a step away from learning the truth.
odpowiedz llm:  Zdanie po an

In [28]:
sampled_df = df.sample(n=10, random_state=50, ignore_index=True)

for index, row in sampled_df.iterrows():
    english_text = row["en_text"]

    prompt_filled = prompt_translation_in_brackets_three_shot.format(en_text=english_text)
    result = llm.prompt_chat_custom_temperature(prompt=prompt_filled, temperature=0.0)
    
    match = re.search(r'\[(.*?)\]', result)
    extracted_translation = None
    if match:
        extracted_translation = match.group(1)
        
    print("zdanie po angielsku: ", english_text)
    print("odpowiedz llm: ", result)
    print("wyciagniete tlumaczenie: " + extracted_translation)
    print("======================================================")
    print()
    
    
    if (index + 1) % 100 == 0:
        print(f"Processed {index + 1} out of {len(sampled_df)} samples...")


zdanie po angielsku:  So be it.
odpowiedz llm:  Zdanie po angielsku: So be it.

Tłumaczenie: [Niech tak będzie.]
wyciagniete tlumaczenie: Niech tak będzie.

zdanie po angielsku:  A year now you've promised to write somethin' for me.
odpowiedz llm:  Tłumaczenie: [Minął już rok odkąd obiecałeś napisać coś dla mnie.]
wyciagniete tlumaczenie: Minął już rok odkąd obiecałeś napisać coś dla mnie.

zdanie po angielsku:  Freya watch over you.
odpowiedz llm:  Oczywiście! Oto tłumaczenie podanego zdania na język polski:

Freya watch over you.

Tłumaczenie: [Freya czuwa nad tobą.]

Warto zauważyć, że w tym kontekście "watch over" może być rozumiane jako "opiekuje się tobą" lub "chroni cię". W języku polskim często używamy zwrotu "czuwa nad tobą", aby wyrazić podobne znaczenie.
wyciagniete tlumaczenie: Freya czuwa nad tobą.

zdanie po angielsku:  You see? We're just a step away from learning the truth.
odpowiedz llm:  Tłumaczenie: [Widzisz? Jesteśmy tylko jeden krok od poznania prawdy.]
wyciagniete

In [14]:
en = "Shaved head and ponytail."

In [22]:
print(llm.prompt_chat_custom_temperature(prompt=prompt_translation_in_brackets_one_shot.format(en_text=en), temperature=0.0))

Zdanie po angielsku: Shaved head and ponytail.

Tłumaczenie: [Gładko ogolona głowa i kucyk.]


In [29]:
print(llm.prompt_chat_custom_temperature(prompt=prompt_translation_in_brackets_three_shot.format(en_text=en), temperature=0.0))

Tłumaczenie: [Golona głowa i kucyk.]


In [30]:
sampled_df = df.sample(n=1000, random_state=50, ignore_index=True)

for index, row in sampled_df.iterrows():
    english_text = row["en_text"]

    prompt_filled_one_shot = prompt_translation_in_brackets_one_shot.format(en_text=english_text)
    result_one_shot = llm.prompt_chat_custom_temperature(prompt=prompt_filled_one_shot, temperature=0.0)

    prompt_filled_three_shot = prompt_translation_in_brackets_three_shot.format(en_text=english_text)
    result_three_shot = llm.prompt_chat_custom_temperature(prompt=prompt_filled_three_shot, temperature=0.0)
    
    match = re.search(r'\[(.*?)\]', result_one_shot)
    extracted_translation_one_shot = None
    if match:
        extracted_translation_one_shot = match.group(1)


    match = re.search(r'\[(.*?)\]', result_three_shot)
    extracted_translation_three_shot = None
    if match:
        extracted_translation_three_shot = match.group(1)
        
    sampled_df.at[index, f'result_one_shot'] = result_one_shot
    sampled_df.at[index, f'translation_one_shot'] = extracted_translation_one_shot
    sampled_df.at[index, f'result_three_shot'] = result_three_shot
    sampled_df.at[index, f'translation_three_shot'] = extracted_translation_three_shot

    
    if (index + 1) % 100 == 0:
        print(f"Processed {index + 1} out of {len(sampled_df)} samples...")

sampled_df.to_csv("results/new_extraction_comparision.csv", sep="|", index=False)

Processed 100 out of 1000 samples...
Processed 200 out of 1000 samples...
Processed 300 out of 1000 samples...
Processed 400 out of 1000 samples...
Processed 500 out of 1000 samples...
Processed 600 out of 1000 samples...
Processed 700 out of 1000 samples...
Processed 800 out of 1000 samples...
Processed 900 out of 1000 samples...
Processed 1000 out of 1000 samples...


In [31]:
new_example = "I been already. They said their herbs can't help, and all that's left is to pray."

In [ ]:
print(llm.prompt_chat_custom_temperature(prompt=prompt_translation_in_brackets_one_shot.format(en_text=new_example), temperature=0.0))

Zdanie po angielsku: I been already. They said their herbs can't help, and all that's left is to pray.

Tłumaczenie: [Już byłem/byłam tutaj.] [Powiedzieli, że ich zioła nie mogą pomóc, i pozostało tylko się modlić.]

Warto zauważyć, że drugie zdanie zawiera kilka błędów gramatycznych i składniowych w oryginalnym tekście. Poprawiona wersja brzmiałaby: "I have been here already. They said their herbs can't help, and all that's left is to pray." Tłumaczenie uwzględniające te poprawki:

[Już tu byłem/byłam.] [Powiedzieli, że ich zioła nie mogą pomóc, i pozostało tylko się modlić.]


SPRAWDZIC TO SAMO ZDANIE DLA 3-shot